In [1]:
import pandas as pd
import json

In [2]:
material_data = pd.read_csv("result_data/easy_material_perf_10.csv")

social_data = pd.read_csv("result_data/easy_social_perf_10.csv")

physical_data = pd.read_csv("result_data/easy_physical_perf_10.csv")

In [3]:
len(physical_data[physical_data["avg_ratio_score"]==-1.0])

0

In [4]:
material_data[(material_data["avg_binary_score"]==1.0) | (material_data["avg_binary_score"]==0.0)]

,truism_number,perturbation,premise,avg_binary_score,avg_ratio_score
0,0,original,original,1.0,0.897700
1,1,original,original,1.0,0.638536
2,2,original,original,1.0,0.768453
3,3,original,original,1.0,0.562546
4,4,original,original,1.0,0.646663
5,5,original,original,1.0,0.466518
6,6,original,original,1.0,0.761342
8,8,original,original,1.0,0.812121
10,10,original,original,0.0,-0.277782
11,11,original,original,1.0,0.986556


In [5]:
physical_data[(physical_data["avg_binary_score"]==1.0) | (physical_data["avg_binary_score"]==0.0)]

,truism_number,perturbation,premise,avg_binary_score,avg_ratio_score
0,0,original,original,1.0,0.973197
1,1,original,original,1.0,0.801318
2,2,original,original,1.0,0.954682
3,3,original,original,1.0,0.602176
4,4,original,original,1.0,0.912208
5,5,original,original,1.0,0.982812
6,6,original,original,1.0,0.780750
7,7,original,original,1.0,0.565377
8,8,original,original,1.0,0.877403
9,9,original,original,1.0,0.789617


In [6]:
social_data[(social_data["avg_binary_score"]==1.0) | (social_data["avg_binary_score"]==0.0)]

,truism_number,perturbation,premise,avg_binary_score,avg_ratio_score
0,0,original,original,1.0,0.825030
1,1,original,original,1.0,0.865819
2,2,original,original,1.0,0.946480
3,3,original,original,1.0,0.951042
4,4,original,original,1.0,0.552983
5,5,original,original,1.0,0.918058
6,6,original,original,1.0,0.857194
7,7,original,original,1.0,0.918085
8,8,original,original,1.0,0.997685
9,9,original,original,1.0,0.980260


In [58]:
with open("truism_data/physical_data_2.json", "r") as f:
    physical_params = json.load(f)

with open("truism_data/material_data_2.json", "r") as f:
    material_params = json.load(f)

with open("truism_data/social_data_2.json", "r") as f:
    social_params = json.load(f)

In [59]:
physical_params["0"]

{'initial_comparison': 'A is smaller than B',
 'negation_switch': {'0': ['is', 'is not'],
  '1': ['can', 'can not'],
  '2': ['finds', 'does not find']},
 'antonym_switch': ['put into a box', 'contain an object'],
 'original_comparison': 'is easier to put into a box than',
 'paraphrase': 'can fit into more containers than',
 'inverted_paraphrase': 'finds it harder to contain objects than',
 'premise_switch': {'0': ['easier', 'harder'],
  '1': ['more', 'less'],
  '2': ['harder', 'easier']},
 'template': '4',
 'is_more': True,
 'greater_than': 'A'}

In [60]:
def get_templates(param_array):
    templates = {}
    for param_dict in param_array:
        for key in param_dict:
            templates[param_dict[key]["template"]] = {}
    return templates

In [61]:
templates = get_templates([physical_params, material_params, social_params])

In [62]:
def get_perturbations(result_data, template_dict):
#     print(result_data)
    for i, row in result_data.iterrows():
#         print(row)
        p_key = row["perturbation"] + "-" + row["premise"]
#         print(p_key)
        for key in template_dict:
            template_dict[key][p_key] = {"accuracy" : 0, "ratio_score" : 0}
    return template_dict

In [63]:
filled_templates = get_perturbations(material_data, templates)

In [64]:
def get_perturbation_order(result_data, loop_count):
    order = []
    for i, row in result_data.iterrows():
        p_key = row["perturbation"] + "-" + row["premise"]
        order.append(p_key)
        if i >= loop_count-1:
            break
    
    return order
        

In [65]:
perturbation_order = get_perturbation_order(material_data, 24)

In [66]:
for key in filled_templates:
    filled_templates[key]["count"] = 0

In [67]:
def fill_pertubation_data(result_data, param_data, template_dict, multiple=1):
    completed_truisms = {}
    
    for i, row in result_data.iterrows():
        p_key = row["perturbation"] + "-" + row["premise"]
        template_key = param_data[str(row["truism_number"])]["template"]
        
        template_dict[template_key][p_key]["accuracy"] += row["avg_binary_score"]*multiple
        template_dict[template_key][p_key]["ratio_score"] += row["avg_ratio_score"]*multiple
        
        if row["truism_number"] not in completed_truisms:
            template_dict[template_key]["count"] += 1*multiple
            completed_truisms[row["truism_number"]] = 1

    return template_dict

In [68]:
filled_templates = fill_pertubation_data(material_data, material_params, filled_templates)
filled_templates = fill_pertubation_data(physical_data, physical_params, filled_templates)
filled_templates = fill_pertubation_data(social_data, social_params, filled_templates)

In [69]:
filled_templates["1"]

{'original-original': {'accuracy': 12.0, 'ratio_score': 1.4541697180597146},
 'original-asymmetric_premise': {'accuracy': 11.0,
  'ratio_score': 0.3095939060042199},
 'original-asymmetric_conclusion': {'accuracy': 11.0,
  'ratio_score': -0.2720905529195701},
 'negation-original': {'accuracy': 10.0, 'ratio_score': -0.4219752981583015},
 'negation-asymmetric_premise': {'accuracy': 10.0,
  'ratio_score': 0.38345696584266287},
 'negation-asymmetric_conclusion': {'accuracy': 10.0,
  'ratio_score': 0.3516560107965965},
 'antonym-original': {'accuracy': 10.0, 'ratio_score': -1.4265330327784738},
 'antonym-asymmetric_premise': {'accuracy': 10.0,
  'ratio_score': 0.5028927615624144},
 'antonym-asymmetric_conclusion': {'accuracy': 10.0,
  'ratio_score': 0.6634761586965511},
 'paraphrase-original': {'accuracy': 11.0, 'ratio_score': 2.2874458349832225},
 'paraphrase-asymmetric_premise': {'accuracy': 10.0,
  'ratio_score': -0.22225789746512326},
 'paraphrase-asymmetric_conclusion': {'accuracy': 9.0

In [70]:
averaged_acc_templates = {}
averaged_ratio_templates = {}
for template in filled_templates:
    averaged_acc_templates[template] = {}
    averaged_ratio_templates[template] = {}
    count = float(filled_templates[template]["count"])
    for perturbation in filled_templates[template]:
        if perturbation != "count":
            acc = filled_templates[template][perturbation]["accuracy"]
            score = filled_templates[template][perturbation]["ratio_score"]

            averaged_acc_templates[template][perturbation] = acc / count
            averaged_ratio_templates[template][perturbation] = score/count

In [71]:
averaged_acc_templates["1"]

{'original-original': 0.6,
 'original-asymmetric_premise': 0.55,
 'original-asymmetric_conclusion': 0.55,
 'negation-original': 0.5,
 'negation-asymmetric_premise': 0.5,
 'negation-asymmetric_conclusion': 0.5,
 'antonym-original': 0.5,
 'antonym-asymmetric_premise': 0.5,
 'antonym-asymmetric_conclusion': 0.5,
 'paraphrase-original': 0.55,
 'paraphrase-asymmetric_premise': 0.5,
 'paraphrase-asymmetric_conclusion': 0.45,
 'paraphrase_inversion-original': 0.6,
 'paraphrase_inversion-asymmetric_premise': 0.35,
 'paraphrase_inversion-asymmetric_conclusion': 0.35,
 'negation_antonym-original': 0.5,
 'negation_antonym-asymmetric_premise': 0.45,
 'negation_antonym-asymmetric_conclusion': 0.45,
 'negation_paraphrase-original': 0.4,
 'negation_paraphrase-asymmetric_premise': 0.6,
 'negation_paraphrase-asymmetric_conclusion': 0.6,
 'negation_paraphrase_inversion-original': 0.35,
 'negation_paraphrase_inversion-asymmetric_premise': 0.6,
 'negation_paraphrase_inversion-asymmetric_conclusion': 0.55}

In [72]:
averaged_ratio_templates["1"]

{'original-original': 0.07270848590298573,
 'original-asymmetric_premise': 0.015479695300210993,
 'original-asymmetric_conclusion': -0.013604527645978503,
 'negation-original': -0.021098764907915075,
 'negation-asymmetric_premise': 0.019172848292133143,
 'negation-asymmetric_conclusion': 0.017582800539829822,
 'antonym-original': -0.07132665163892368,
 'antonym-asymmetric_premise': 0.02514463807812072,
 'antonym-asymmetric_conclusion': 0.033173807934827555,
 'paraphrase-original': 0.11437229174916112,
 'paraphrase-asymmetric_premise': -0.011112894873256163,
 'paraphrase-asymmetric_conclusion': -0.0472725899932715,
 'paraphrase_inversion-original': 0.2070199929829037,
 'paraphrase_inversion-asymmetric_premise': -0.22276449427825568,
 'paraphrase_inversion-asymmetric_conclusion': -0.22613121798396452,
 'negation_antonym-original': 0.09888045730052841,
 'negation_antonym-asymmetric_premise': -0.12375429222736718,
 'negation_antonym-asymmetric_conclusion': -0.10537622840592267,
 'negation_

In [73]:
# physical_data[physical_data["truism_number"] == 1]

In [74]:
def create_tables(averaged_numbers, perturbation_order, threshold):
    output = {}
    output_2 = {}
    sorted_keys = list(averaged_numbers.keys())
    sorted_keys.sort()
    for template in sorted_keys:
        for perturbation in perturbation_order:
            if perturbation in output:
                output[perturbation].append(averaged_numbers[template][perturbation])
                if averaged_numbers[template][perturbation] <= threshold:
                    output_2[perturbation].append("X")
                else:
                    output_2[perturbation].append("O")
            else:
                output[perturbation] = []
                output_2[perturbation] = []
                output[perturbation].append(averaged_numbers[template][perturbation])
                if averaged_numbers[template][perturbation] < threshold:
                    output_2[perturbation].append("X")
                elif averaged_numbers[template][perturbation] == threshold:
                    output_2[perturbation].append("---")
                else:
                    output_2[perturbation].append("O")
    
    return (pd.DataFrame.from_dict(output), pd.DataFrame.from_dict(output_2))

In [75]:
accuracy_table, visual_table = create_tables(averaged_acc_templates, perturbation_order, 0.5)


In [76]:
accuracy_table

,original-original,original-asymmetric_premise,original-asymmetric_conclusion,negation-original,negation-asymmetric_premise,negation-asymmetric_conclusion,antonym-original,antonym-asymmetric_premise,antonym-asymmetric_conclusion,paraphrase-original,...,paraphrase_inversion-asymmetric_conclusion,negation_antonym-original,negation_antonym-asymmetric_premise,negation_antonym-asymmetric_conclusion,negation_paraphrase-original,negation_paraphrase-asymmetric_premise,negation_paraphrase-asymmetric_conclusion,negation_paraphrase_inversion-original,negation_paraphrase_inversion-asymmetric_premise,negation_paraphrase_inversion-asymmetric_conclusion
0,0.600000,0.550000,0.550000,0.500000,0.5,0.5,0.5,0.500000,0.500000,0.550000,...,0.350000,0.5,0.45,0.45,0.400000,0.600000,0.600000,0.35,0.6,0.550000
1,0.500000,0.500000,0.500000,0.500000,0.4,0.4,0.5,0.400000,0.400000,0.500000,...,0.500000,0.5,0.50,0.50,0.400000,0.500000,0.500000,0.50,0.4,0.400000
2,0.500000,0.500000,0.500000,0.750000,0.5,0.5,0.5,0.750000,0.750000,0.500000,...,0.500000,0.5,0.50,0.50,0.500000,0.500000,0.500000,0.50,0.5,0.500000
3,0.615385,0.461538,0.461538,0.615385,0.5,0.5,0.5,0.538462,0.538462,0.653846,...,0.538462,0.5,0.50,0.50,0.615385,0.384615,0.384615,0.50,0.5,0.461538


In [77]:
visual_table

,original-original,original-asymmetric_premise,original-asymmetric_conclusion,negation-original,negation-asymmetric_premise,negation-asymmetric_conclusion,antonym-original,antonym-asymmetric_premise,antonym-asymmetric_conclusion,paraphrase-original,...,paraphrase_inversion-asymmetric_conclusion,negation_antonym-original,negation_antonym-asymmetric_premise,negation_antonym-asymmetric_conclusion,negation_paraphrase-original,negation_paraphrase-asymmetric_premise,negation_paraphrase-asymmetric_conclusion,negation_paraphrase_inversion-original,negation_paraphrase_inversion-asymmetric_premise,negation_paraphrase_inversion-asymmetric_conclusion
0,O,O,O,---,---,---,---,---,---,O,...,X,---,X,X,X,O,O,X,O,O
1,X,X,X,X,X,X,X,X,X,X,...,X,X,X,X,X,X,X,X,X,X
2,X,X,X,O,X,X,X,O,O,X,...,X,X,X,X,X,X,X,X,X,X
3,O,X,X,O,X,X,X,O,O,O,...,O,X,X,X,O,X,X,X,X,X


In [79]:
pd.DataFrame.to_csv(accuracy_table, "visual_data_two/accuracy_table_2.csv", index=False)
pd.DataFrame.to_csv(visual_table, "visual_data_two/acc_visual_table_2.csv", index=False)

In [80]:
ratio_table, visual_table = create_tables(averaged_ratio_templates, perturbation_order, 0)


In [81]:
pd.DataFrame.to_csv(ratio_table, "visual_data_two/ratio_table_2.csv", index=False)
pd.DataFrame.to_csv(visual_table, "visual_data_two/ratio_visual_table_2.csv", index=False)

In [82]:
def aggregate_pertubations(raw_template_data, metric):
    total_count = 0.0
    for template in raw_template_data:
        total_count += raw_template_data[template]["count"]
    
    one_key = list(raw_template_data.keys())[0]
    
    output = {}
    for perturbation in raw_template_data[one_key]:
        if perturbation != "count":
            for template in raw_template_data:
                if perturbation in output:
                    output[perturbation] += raw_template_data[template][perturbation][metric]
                else:
                    output[perturbation] = raw_template_data[template][perturbation][metric]
    
    for key in output:
        output[key] = output[key] / total_count
    
    return output

In [83]:
aggregate_pertubations(filled_templates, "accuracy")

{'original-original': 0.5833333333333334,
 'original-asymmetric_premise': 0.5,
 'original-asymmetric_conclusion': 0.5,
 'negation-original': 0.5666666666666667,
 'negation-asymmetric_premise': 0.48333333333333334,
 'negation-asymmetric_conclusion': 0.48333333333333334,
 'antonym-original': 0.5,
 'antonym-asymmetric_premise': 0.5166666666666667,
 'antonym-asymmetric_conclusion': 0.5166666666666667,
 'paraphrase-original': 0.5833333333333334,
 'paraphrase-asymmetric_premise': 0.5166666666666667,
 'paraphrase-asymmetric_conclusion': 0.5,
 'paraphrase_inversion-original': 0.5166666666666667,
 'paraphrase_inversion-asymmetric_premise': 0.4666666666666667,
 'paraphrase_inversion-asymmetric_conclusion': 0.4666666666666667,
 'negation_antonym-original': 0.5,
 'negation_antonym-asymmetric_premise': 0.48333333333333334,
 'negation_antonym-asymmetric_conclusion': 0.48333333333333334,
 'negation_paraphrase-original': 0.5,
 'negation_paraphrase-asymmetric_premise': 0.48333333333333334,
 'negation_p

In [84]:
def aggregate_templates(raw_template_data, metric):   
    output = {}
    for template in raw_template_data:
        output[template] = {"count" : 0, "pct" : -1, "total" : -1}
        for perturbation in raw_template_data[template]:
            if perturbation != "count":
#                 if template in output:
                output[template]["count"] += raw_template_data[template][perturbation][metric]
#                 else:
#                     output[template] = raw_template_data[template][perturbation][metric]
    
    for key in output:
        output[key]["total"] = float((len(raw_template_data[key].keys()) - 1) * raw_template_data[key]["count"])
        output[key]["pct"] = output[key]["count"] / output[key]["total"]
    
    return output

In [85]:
aggregate_templates(filled_templates, "accuracy")

{'4': {'count': 324.0, 'pct': 0.5192307692307693, 'total': 624.0},
 '1': {'count': 240.0, 'pct': 0.5, 'total': 480.0},
 '3': {'count': 52.0, 'pct': 0.5416666666666666, 'total': 96.0},
 '2': {'count': 109.0, 'pct': 0.45416666666666666, 'total': 240.0}}